In [2]:
prompt = """
You are an expert data scientist currently applying for the job of the senior AI research scientist. This job is offered by Infobip, one of the most popular omnichannel marketing platforms. You passed the initial recruitment call and technical interview. After the technical interview, you got the recruitment assignment. The research scenario to be conducted is summarized as follows:
* there are multiple e-commerce clients in South East Asia (possibly dealing with different regional languages)
* the clients use telecommunication APIs to send mass-scale newsletters via e-mail
* each newsletter campaign reaches hundreds of thousands end-users, which may or may not interact with the products advertised in those campaigns
* some clients send newsletters with links to the products where you can track clicks, whereas other clients don’t (static catalogues)
* some of those clients (probably more those without trackable click system, but include both) need help to boost their revenue using the simple data
The task is the following:
* build a conversational recommender system (e.g., as a WhatsApp chatbot) for up-selling and cross-selling of products that a specific client advertise
* leverage the available historical newsletter data
* assume there is existing system available to deploy a conversational agent (e.g., a WhatsApp chatbot) and develop only the necessary APIs that take a textual input in and can return any kind of output (e.g., text, image, etc.)
Please, create this conversational recommender system and describe its architecture to answer the following:
* What specific components (e.g., data pipelines, NLP extraction methods, embedding models, generative AI models, recommendation algorithms) are necessary?
* Which NLP tools or models are suitable for multilingual processing, particularly for Chinese?
* How will the data extraction system integrate with a generative conversational recommender system (recommendation logic, personalization capabilities, conversational flow)?
Additional notes. Use most state of the art techniques. Use web scraping tools if possible. Use LangGraph to construct the agentic AI workflow. Use multilingual models or models finetuned on Chinese. Do evaluation of both LLM, agent and the overall system against some kind of golden data. Allow the agent to have memory in order to have a fluid conversation. Use evals or judges if necessary. Avoid using human in the loop. Document the process of building this system. Make sure the code is runnable from within the jupyter notebook.
"""

In [ ]:
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic

In [4]:
load_dotenv()

True

In [17]:
llm = ChatAnthropic(
    model="claude-opus-4-20250514",
    temperature=0.7,
    timeout=None,
)

In [19]:
res = llm.invoke(prompt, max_tokens=32000)

In [20]:
print(res.content)

I'll create a comprehensive conversational recommender system for Infobip's e-commerce clients in Southeast Asia. Let me design and implement this system step by step.

```python
# Install required packages
!pip install -q langchain langgraph chromadb sentence-transformers openai anthropic
!pip install -q beautifulsoup4 requests pandas numpy scikit-learn
!pip install -q transformers torch torchvision
!pip install -q plotly matplotlib seaborn
!pip install -q pydantic typing-extensions
!pip install -q faiss-cpu
!pip install -q jieba pythainlp underthesea
!pip install -q evaluate rouge_score bert_score
```

## 1. System Architecture Overview

```python
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
from enum import Enum

# Core ML/NLP libraries
import torch
from transformers import (
    AutoTokenizer, 
    AutoModel, 
    AutoModelForSequenceClassif

In [ ]:
## 1. System Architecture Overview

```python
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
from enum import Enum

# Core ML/NLP libraries
import torch
from transformers import (
    AutoTokenizer, 
    AutoModel, 
    AutoModelForSequenceClassification,
    pipeline
)
from sentence_transformers import SentenceTransformer

# LangChain and LangGraph
from langchain.chat_models import ChatOpenAI, ChatAnthropic
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import Document
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor, ToolInvocation

# Web scraping
from bs4 import BeautifulSoup
import requests

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Set up environment
os.environ["OPENAI_API_KEY"] = "your-api-key"  # Replace with actual key
os.environ["ANTHROPIC_API_KEY"] = "your-api-key"  # Replace with actual key

## 2. Data Models and Structures

@dataclass
class Product:
    """Product data model"""
    id: str
    name: str
    description: str
    price: float
    category: str
    subcategory: Optional[str]
    brand: Optional[str]
    image_url: Optional[str]
    attributes: Dict[str, Any]
    language: str
    
@dataclass
class UserInteraction:
    """User interaction data model"""
    user_id: str
    product_id: str
    interaction_type: str  # 'click', 'view', 'purchase'
    timestamp: datetime
    newsletter_id: Optional[str]
    
@dataclass
class NewsletterCampaign:
    """Newsletter campaign data model"""
    campaign_id: str
    client_id: str
    sent_date: datetime
    products: List[str]
    has_tracking: bool
    language: str
    
class ConversationState(Enum):
    """Conversation states for the chatbot"""
    GREETING = "greeting"
    PRODUCT_DISCOVERY = "product_discovery"
    RECOMMENDATION = "recommendation"
    CLARIFICATION = "clarification"
    CHECKOUT = "checkout"
    FEEDBACK = "feedback"

## 3. Multilingual NLP Components

class MultilingualNLPProcessor:
    """Handles multilingual text processing with focus on Southeast Asian languages"""
    
    def __init__(self):
        # Load multilingual models
        self.embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
        self.classifier = pipeline(
            "zero-shot-classification",
            model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
        )
        
        # Language-specific tokenizers
        self.tokenizers = {
            'zh': None,  # Will use jieba
            'th': None,  # Will use pythainlp
            'vi': None,  # Will use underthesea
            'en': None   # Default tokenizer
        }
        
    def detect_language(self, text: str) -> str:
        """Detect language of input text"""
        from langdetect import detect
        try:
            return detect(text)
        except:
            return 'en'
    
    def tokenize(self, text: str, lang: str) -> List[str]:
        """Language-specific tokenization"""
        if lang == 'zh':
            import jieba
            return list(jieba.cut(text))
        elif lang == 'th':
            from pythainlp.tokenize import word_tokenize
            return word_tokenize(text)
        elif lang == 'vi':
            from underthesea import word_tokenize
            return word_tokenize(text)
        else:
            return text.split()
    
    def extract_entities(self, text: str, lang: str) -> Dict[str, List[str]]:
        """Extract named entities from text"""
        # Use multilingual NER model
        ner_pipeline = pipeline(
            "ner", 
            model="Davlan/xlm-roberta-large-ner-hrl",
            aggregation_strategy="simple"
        )
        entities = ner_pipeline(text)
        
        # Group by entity type
        grouped_entities = {}
        for ent in entities:
            ent_type = ent['entity_group']
            if ent_type not in grouped_entities:
                grouped_entities[ent_type] = []
            grouped_entities[ent_type].append(ent['word'])
            
        return grouped_entities
    
    def classify_intent(self, text: str, candidate_labels: List[str]) -> Dict[str, float]:
        """Classify user intent"""
        result = self.classifier(text, candidate_labels)
        return dict(zip(result['labels'], result['scores']))
    
    def embed_text(self, texts: List[str]) -> np.ndarray:
        """Generate embeddings for texts"""
        return self.embedder.encode(texts, convert_to_numpy=True)

## 4. Web Scraping Component

```python
class ProductWebScraper:
    """Scrapes product information from e-commerce websites"""
    
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
    def scrape_product_page(self, url: str) -> Optional[Dict[str, Any]]:
        """Scrape product information from a given URL"""
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Generic extraction patterns (would be customized per site)
            product_data = {
                'url': url,
                'title': self._extract_title(soup),
                'price': self._extract_price(soup),
                'description': self._extract_description(soup),
                'images': self._extract_images(soup),
                'attributes': self._extract_attributes(soup)
            }
            
            return product_data
        except Exception as e:
            print(f"Error scraping {url}: {str(e)}")
            return None
    
    def _extract_title(self, soup: BeautifulSoup) -> str:
        """Extract product title"""
        # Try common patterns
        patterns = [
            ('h1', {'class': 'product-title'}),
            ('h1', {'itemprop': 'name'}),
            ('h1', {}),
            ('title', {})
        ]
        
        for tag, attrs in patterns:
            element = soup.find(tag, attrs)
            if element:
                return element.get_text(strip=True)
        return ""
    
    def _extract_price(self, soup: BeautifulSoup) -> Optional[float]:
        """Extract product price"""
        patterns = [
            ('span', {'class': 'price'}),
            ('span', {'itemprop': 'price'}),
            ('meta', {'property': 'product:price:amount'})
        ]
        
        for tag, attrs in patterns:
            element = soup.find(tag, attrs)
            if element:
                price_text = element.get_text(strip=True) if tag != 'meta' else element.get('content')
                # Extract numeric value
                import re
                numbers = re.findall(r'[\d,]+\.?\d*', price_text)
                if numbers:
                    return float(numbers[0].replace(',', ''))
        return None
    
    def _extract_description(self, soup: BeautifulSoup) -> str:
        """Extract product description"""
        patterns = [
            ('div', {'class': 'product-description'}),
            ('div', {'itemprop': 'description'}),
            ('section', {'class': 'description'})
        ]
        
        for tag, attrs in patterns:
            element = soup.find(tag, attrs)
            if element:
                return element.get_text(strip=True)
        return ""
    
    def _extract_images(self, soup: BeautifulSoup) -> List[str]:
        """Extract product images"""
        images = []
        
        # Try different image patterns
        img_elements = soup.find_all('img', {'class': re.compile('product|gallery')})
        for img in img_elements[:5]:  # Limit to 5 images
            src = img.get('src') or img.get('data-src')
            if src:
                images.append(src)
                
        return images
    
    def _extract_attributes(self, soup: BeautifulSoup) -> Dict[str, str]:
        """Extract product attributes"""
        attributes = {}
        
        # Look for specification tables
        spec_table = soup.find('table', {'class': re.compile('spec|attribute')})
        if spec_table:
            rows = spec_table.find_all('tr')
            for row in rows:
                cells = row.find_all(['td', 'th'])
                if len(cells) >= 2:
                    key = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True)
                    attributes[key] = value
                    
        return attributes

## 5. Recommendation Engine

```python
class HybridRecommendationEngine:
    """Hybrid recommendation system combining collaborative and content-based filtering"""
    
    def __init__(self, nlp_processor: MultilingualNLPProcessor):
        self.nlp_processor = nlp_processor
        self.product_embeddings = None
        self.user_embeddings = None
        self.interaction_matrix = None
        
    def build_product_embeddings(self, products: List[Product]) -> None:
        """Build product embeddings from descriptions"""
        texts = []
        self.product_ids = []
        
        for product in products:
            # Combine product attributes for embedding
            text = f"{product.name} {product.description} {product.category}"
            if product.brand:
                text += f" {product.brand}"
            texts.append(text)
            self.product_ids.append(product.id)
            
        self.product_embeddings = self.nlp_processor.embed_text(texts)
        
        # Create FAISS index for fast similarity search
        import faiss
        self.product_index = faiss.IndexFlatL2(self.product_embeddings.shape[1])
        self.product_index.add(self.product_embeddings)
        
    def build_user_profiles(self, interactions: List[UserInteraction]) -> None:
        """Build user profiles from interaction history"""
        from collections import defaultdict
        
        # Create user-item interaction matrix
        user_items = defaultdict(lambda: defaultdict(float))
        
        for interaction in interactions:
            # Weight interactions by type
            weight = {
                'view': 1.0,
                'click': 2.0,
                'purchase': 5.0
            }.get(interaction.interaction_type, 1.0)
            
            user_items[interaction.user_id][interaction.product_id] += weight
            
        # Convert to matrix format
        users = sorted(user_items.keys())
        items = sorted(set(item for user_dict in user_items.values() for item in user_dict))
        
        self.interaction_matrix = np.zeros((len(users), len(items)))
        self.user_id_map = {uid: idx for idx, uid in enumerate(users)}
        self.item_id_map = {iid: idx for idx, iid in enumerate(items)}
        
        for user_id, user_dict in user_items.items():
            user_idx = self.user_id_map[user_id]
            for item_id, score in user_dict.items():
                if item_id in self.item_id_map:
                    item_idx = self.item_id_map[item_id]
                    self.interaction_matrix[user_idx, item_idx] = score
                    
    def get_content_based_recommendations(
        self, 
        product_id: str, 
        n_recommendations: int = 5
    ) -> List[Tuple[str, float]]:
        """Get content-based recommendations"""
        if product_id not in self.product_ids:
            return []
            
        # Get product embedding
        idx = self.product_ids.index(product_id)
        query_embedding = self.product_embeddings[idx:idx+1]
        
        # Find similar products
        distances, indices = self.product_index.search(query_embedding, n_recommendations + 1)
        
        recommendations = []
        for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
            if i == 0:  # Skip the query product itself
                continue
            rec_product_id = self.product_ids[idx]
            similarity = 1 / (1 + dist)  # Convert distance to similarity
            recommendations.append((rec_product_id, similarity))
            
        return recommendations[:n_recommendations]
    
    def get_collaborative_recommendations(
        self, 
        user_id: str, 
        n_recommendations: int = 5
    ) -> List[Tuple[str, float]]:
        """Get collaborative filtering recommendations"""
        if user_id not in self.user_id_map:
            return []
            
        # Use SVD for collaborative filtering
        from sklearn.decomposition import TruncatedSVD
        
        svd = TruncatedSVD(n_components=min(50, self.interaction_matrix.shape[1] - 1))
        user_factors = svd.fit_transform(self.interaction_matrix)
        item_factors = svd.components_.T
        
        # Get user vector
        user_idx = self.user_id_map[user_id]
        user_vec = user_factors[user_idx]
        
        # Calculate scores for all items
        scores = np.dot(item_factors, user_vec)
        
        # Get top recommendations
        item_indices = np.argsort(scores)[::-1]
        
        recommendations = []
        reverse_item_map = {v: k for k, v in self.item_id_map.items()}
        
        for idx in item_indices[:n_recommendations]:
            if idx in reverse_item_map:
                item_id = reverse_item_map[idx]
                score = scores[idx]
                recommendations.append((item_id, score))
                
        return recommendations
    
    def get_hybrid_recommendations(
        self,
        user_id: Optional[str] = None,
        product_id: Optional[str] = None,
        n_recommendations: int = 5,
        alpha: float = 0.5
    ) -> List[Tuple[str, float]]:
        """Get hybrid recommendations combining both approaches"""
        recommendations = {}
        
        # Get content-based recommendations if product_id provided
        if product_id:
            content_recs = self.get_content_based_recommendations(product_id, n_recommendations * 2)
            for prod_id, score in content_recs:
                recommendations[prod_id] = score * (1 - alpha)
                
        # Get collaborative recommendations if user_id provided
        if user_id:
            collab_recs = self.get_collaborative_recommendations(user_id, n_recommendations * 2)
            for prod_id, score in collab_recs:
                if prod_id in recommendations:
                    recommendations[prod_id] += score * alpha
                else:
                    recommendations[prod_id] = score * alpha
                    
        # Sort and return top recommendations
        sorted_recs = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
        return sorted_recs[:n_recommendations]

## 6. Conversational Agent with LangGraph

from typing import TypedDict, Annotated, Sequence
from langchain.tools import tool
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """State for the conversational agent"""
    messages: Annotated[Sequence, add_messages]
    user_id: str
    current_product: Optional[str]
    conversation_state: str
    language: str
    recommendations: List[Dict[str, Any]]
    user_preferences: Dict[str, Any]

class ConversationalRecommenderAgent:
    """Main conversational agent using LangGraph"""
    
    def __init__(
        self,
        nlp_processor: MultilingualNLPProcessor,
        recommendation_engine: HybridRecommendationEngine,
        product_catalog: Dict[str, Product]
    ):
        self.nlp_processor = nlp_processor
        self.recommendation_engine = recommendation_engine
        self.product_catalog = product_catalog
        
        # Initialize LLM
        self.llm = ChatOpenAI(
            model="gpt-4",
            temperature=0.7
        )
        
        # Initialize memory
        self.memory = ConversationBufferWindowMemory(
            k=10,  # Keep last 10 exchanges
            return_messages=True
        )
        
        # Build the graph
        self.graph = self._build_graph()
        
    def _build_graph(self) -> StateGraph:
        """Build the LangGraph workflow"""
        workflow = StateGraph(AgentState)
        
        # Define nodes
        workflow.add_node("language_detection", self.detect_language_node)
        workflow.add_node("intent_classification", self.classify_intent_node)
        workflow.add_node("entity_extraction", self.extract_entities_node)
        workflow.add_node("generate_recommendations", self.generate_recommendations_node)
        workflow.add_node("format_response", self.format_response_node)
        workflow.add_node("update_preferences", self.update_preferences_node)
        
        # Define edges
        workflow.set_entry_point("language_detection")
        
        workflow.add_edge("language_detection", "intent_classification")
        workflow.add_edge("intent_classification", "entity_extraction")
        workflow.add_edge("entity_extraction", "generate_recommendations")
        workflow.add_edge("generate_recommendations", "format_response")
        workflow.add_edge("format_response", "update_preferences")
        workflow.add_edge("update_preferences", END)
        
        return workflow.compile()
    
    def detect_language_node(self, state: AgentState) -> AgentState:
        """Detect language of the user message"""
        last_message = state["messages"][-1]
        language = self.nlp_processor.detect_language(last_message.content)
        state["language"] = language
        return state
    
    def classify_intent_node(self, state: AgentState) -> AgentState:
        """Classify user intent"""
        last_message = state["messages"][-1]
        
        intent_labels = [
            "product_search",
            "ask_recommendation",
            "product_comparison",
            "price_inquiry",
            "availability_check",
            "general_question",
            "feedback",
            "greeting"
        ]
        
        intents = self.nlp_processor.classify_intent(
            last_message.content,
            intent_labels
        )
        
        # Update conversation state based on intent
        top_intent = max(intents.items(), key=lambda x: x[1])[0]
        
        state_mapping = {
            "greeting": ConversationState.GREETING.value,
            "product_search": ConversationState.PRODUCT_DISCOVERY.value,
            "ask_recommendation": ConversationState.RECOMMENDATION.value,
            "feedback": ConversationState.FEEDBACK.value
        }
        
        state["conversation_state"] = state_mapping.get(
            top_intent, 
            ConversationState.CLARIFICATION.value
        )
        
        return state
    
    def extract_entities_node(self, state: AgentState) -> AgentState:
        """Extract entities from user message"""
        last_message = state["messages"][-1]
        entities = self.nlp_processor.extract_entities(
            last_message.content,
            state["language"]
        )
        
        # Update user preferences based on entities
        if "user_preferences" not in state:
            state["user_preferences"] = {}
            
        # Extract product categories, brands, price ranges, etc.
        if "ORG" in entities:  # Organizations often indicate brands
            state["user_preferences"]["brands"] = entities["ORG"]
            
        if "MONEY" in entities:  # Price preferences
            state["user_preferences"]["price_range"] = entities["MONEY"]
            
        return state
    
    def generate_recommendations_node(self, state: AgentState) -> AgentState:
        """Generate product recommendations"""
        recommendations = []
        
        if state["conversation_state"] == ConversationState.RECOMMENDATION.value:
            # Get hybrid recommendations
            recs = self.recommendation_engine.get_hybrid_recommendations(
                user_id=state.get("user_id"),
                product_id=state.get("current_product"),
                n_recommendations=5
            )
            
            # Enrich with product details
            for product_id, score in recs:
                if product_id in self.product_catalog:
                    product = self.product_catalog[product_id]
                    recommendations.append({
                        "product": product,
                        "score": score,
                        "reason": self._generate_recommendation_reason(product, state)
                    })
                    
        state["recommendations"] = recommendations
        return state
    
    def _generate_recommendation_reason(
        self, 
        product: Product, 
        state: AgentState
    ) -> str:
        """Generate explanation for why product is recommended"""
        reasons = []
        
        if "brands" in state.get("user_preferences", {}) and \
           product.brand in state["user_preferences"]["brands"]:
            reasons.append(f"from your preferred brand {product.brand}")
            
        if state.get("current_product"):
            reasons.append("similar to what you viewed")
            
        if not reasons:
            reasons.append("popular among similar users")
            
        return " and ".join(reasons)
    
    def format_response_node(self, state: AgentState) -> AgentState:
        """Format the response based on language and context"""
        response_template = self._get_response_template(state)
        
        # Use LLM to generate natural response
        prompt = f"""
        Generate a conversational response in {state['language']} language.
        Context: {state['conversation_state']}
        Recommendations: {json.dumps(state.get('recommendations', []), ensure_ascii=False)}
        Template: {response_template}
        
        Make it natural, friendly, and helpful. Include product details and prices.
        """
        
        response = self.llm.invoke(prompt)
        
        # Add response to messages
        from langchain.schema import AIMessage
        state["messages"].append(AIMessage(content=response.content))
        
        return state
    
    def _get_response_template(self, state: AgentState) -> str:
        """Get response template based on conversation state"""
        templates = {
            ConversationState.GREETING.value: 
                "Welcome! I'm here to help you find great products. What are you looking for today?",
            ConversationState.RECOMMENDATION.value:
                "Based on your preferences, here are my top recommendations: {recommendations}",
            ConversationState.PRODUCT_DISCOVERY.value:
                "I found these products that match your search: {products}",
            ConversationState.CLARIFICATION.value:
                "I'd be happy to help! Could you tell me more about what you're looking for?"
        }
        
        return templates.get(
            state["conversation_state"], 
            templates[ConversationState.CLARIFICATION.value]
        )
    
    def update_preferences_node(self, state: AgentState) -> AgentState:
        """Update user preferences based on conversation"""
        # This would typically update a user profile database
        # For now, we just maintain it in state
        return state
    
    async def process_message(
        self, 
        message: str, 
        user_id: str,
        session_state: Optional[Dict] = None
    ) -> Dict[str, Any]:
        """Process a user message and return response"""
        from langchain.schema import HumanMessage
        
        # Initialize or restore state
        if session_state:
            state = session_state
        else:
            state = {
                "messages": [],
                "user_id": user_id,
                "current_product": None,
                "conversation_state": ConversationState.GREETING.value,
                "language": "en",
                "recommendations": [],
                "user_preferences": {}
            }
            
        # Add user message
        state["messages"].append(HumanMessage(content=message))
        
        # Run the graph
        result = await self.graph.ainvoke(state)
        
        # Extract response
        response = {
            "message": result["messages"][-1].content,
            "recommendations": result.get("recommendations", []),
            "state": result
        }
        
        return response
```

## 7. Evaluation Framework

```python
class RecommenderSystemEvaluator:
    """Comprehensive evaluation system for the recommender"""
    
    def __init__(self):
        self.metrics = {}
        
    def evaluate_recommendations(
        self,
        test_interactions: List[UserInteraction],
        recommendation_engine: HybridRecommendationEngine,
        k: int = 5
    ) -> Dict[str, float]:
        """Evaluate recommendation quality"""
        from sklearn.metrics import precision_score, recall_score, ndcg_score
        
        precisions = []
        recalls = []
        ndcgs = []
        
        # Group interactions by user
        from collections import defaultdict
        user_interactions = defaultdict(list)
        
        for interaction in test_interactions:
            user_interactions[interaction.user_id].append(interaction.product_id)
            
        # Evaluate for each user
        for user_id, true_items in user_interactions.items():
            # Get recommendations
            recs = recommendation_engine.get_hybrid_recommendations(
                user_id=user_id,
                n_recommendations=k
            )
            
            rec_items = [item_id for item_id, _ in recs]
            
            # Calculate metrics
            if rec_items:
                # Precision@k
                hits = len(set(rec_items) & set(true_items))
                precision = hits / len(rec_items)
                precisions.append(precision)
                
                # Recall@k
                recall = hits / len(true_items) if true_items else 0
                recalls.append(recall)
                
                # NDCG@k
                relevance = [1 if item in true_items else 0 for item in rec_items]
                if any(relevance):
                    ndcg = ndcg_score([relevance], [list(range(len(relevance), 0, -1))])
                    ndcgs.append(ndcg)
                    
        return {
            "precision@k": np.mean(precisions),
            "recall@k": np.mean(recalls),
            "ndcg@k": np.mean(ndcgs) if ndcgs else 0.0,
            "coverage": len(set(item for _, items in user_interactions.items() for item in items))
        }
    
    def evaluate_conversation_quality(
        self,
        test_conversations: List[Dict[str, Any]],
        agent: ConversationalRecommenderAgent
    ) -> Dict[str, float]:
        """Evaluate conversation quality using LLM as judge"""
        
        quality_scores = []
        relevance_scores = []
        helpfulness_scores = []
        
        judge_prompt = """
        Evaluate this conversation on a scale of 1-5 for:
        1. Overall Quality
        2. Response Relevance
        3. Helpfulness
        
        Conversation:
        User: {user_message}
        Assistant: {assistant_response}
        
        Provide scores in JSON format: {"quality": X, "relevance": Y, "helpfulness": Z}
        """
        
        judge_llm = ChatOpenAI(model="gpt-4", temperature=0)
        
        for conv in test_conversations:
            # Get judge evaluation
            result = judge_llm.invoke(
                judge_prompt.format(
                    user_message=conv["user_message"],
                    assistant_response=conv["assistant_response"]
                )
            )
            
            try:
                scores = json.loads(result.content)
                quality_scores.append(scores["quality"])
                relevance_scores.append(scores["relevance"])
                helpfulness_scores.append(scores["helpfulness"])
            except:
                pass
                
        return {
            "avg_quality": np.mean(quality_scores),
            "avg_relevance": np.mean(relevance_scores),
            "avg_helpfulness": np.mean(helpfulness_scores)
        }
    
    def evaluate_multilingual_performance(
        self,
        test_texts: Dict[str, List[str]],
        nlp_processor: MultilingualNLPProcessor
    ) -> Dict[str, Dict[str, float]]:
        """Evaluate multilingual NLP performance"""
        results = {}
        
        for lang, texts in test_texts.items():
            # Test language detection accuracy
            correct_detections = sum(
                1 for text in texts 
                if nlp_processor.detect_language(text) == lang
            )
            
            # Test entity extraction (would need ground truth)
            # Test intent classification (would need ground truth)
            
            results[lang] = {
                "language_detection_accuracy": correct_detections / len(texts),
                # Add more metrics as needed
            }
            
        return results
    
    def generate_evaluation_report(
        self,
        all_metrics: Dict[str, Any]
    ) -> None:
        """Generate comprehensive evaluation report"""
        
        # Create visualizations
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # Recommendation metrics
        rec_metrics = all_metrics.get("recommendation_metrics", {})
        axes[0, 0].bar(rec_metrics.keys(), rec_metrics.values())
        axes[0, 0].set_title("Recommendation Performance")
        axes[0, 0].set_ylabel("Score")
        
        # Conversation quality
        conv_metrics = all_metrics.get("conversation_metrics", {})
        axes[0, 1].bar(conv_metrics.keys(), conv_metrics.values())
        axes[0, 1].set_title("Conversation Quality")
        axes[0, 1].set_ylabel("Score (1-5)")
        
        # Language performance
        lang_metrics = all_metrics.get("language_metrics", {})
        if lang_metrics:
            langs = list(lang_metrics.keys())
            accuracies = [lang_metrics[lang]["language_detection_accuracy"] for lang in langs]
            axes[1, 0].bar(langs, accuracies)
            axes[1, 0].set_title("Language Detection Accuracy")
            axes[1, 0].set_ylabel("Accuracy")
            
        plt.tight_layout()
        plt.savefig("evaluation_report.png")
        plt.show()
        
        # Print summary
        print("=== Evaluation Summary ===")
        print(f"Recommendation Precision@5: {rec_metrics.get('precision@k', 0):.3f}")
        print(f"Recommendation Recall@5: {rec_metrics.get('recall@k', 0):.3f}")
        print(f"Conversation Quality: {conv_metrics.get('avg_quality', 0):.2f}/5")
        print(f"Response Relevance: {conv_metrics.get('avg_relevance', 0):.2f}/5")

## 8. Data Pipeline and Integration

```python
class DataPipeline:
    """Handles data ingestion, processing, and integration"""
    
    def __init__(self):
        self.scraper = ProductWebScraper()
        self.nlp_processor = MultilingualNLPProcessor()
        
    def process_newsletter_data(
        self,
        newsletter_file: str
    ) -> Tuple[List[Product], List[NewsletterCampaign]]:
        """Process historical newsletter data"""
        # This would read from actual newsletter data files
        # For demo, we'll create synthetic data
        
        products = []
        campaigns = []
        
        # Simulate processing
        sample_products = [
            {
                "id": "prod_001",
                "name": "智能手机 Pro Max",
                "description": "最新款智能手机，配备先进相机系统",
                "price": 999.99,
                "category": "Electronics",
                "language": "zh"
            },
            {
                "id": "prod_002",
                "name": "Wireless Earbuds",
                "description": "Premium noise-cancelling earbuds",
                "price": 199.99,
                "category": "Electronics",
                "language": "en"
            }
        ]
        
        for prod_data in sample_products:
            product = Product(
                id=prod_data["id"],
                name=prod_data["name"],
                description=prod_data["description"],
                price=prod_data["price"],
                category=prod_data["category"],
                subcategory=None,
                brand="TechBrand",
                image_url=None,
                attributes={},
                language=prod_data["language"]
            )
            products.append(product)
            
        return products, campaigns
    
    def enrich_product_data(
        self,
        products: List[Product],
        scrape_urls: bool = False
    ) -> List[Product]:
        """Enrich product data with web scraping if needed"""
        enriched_products = []
        
        for product in products:
            # If product has URL, scrape additional data
            if scrape_urls and hasattr(product, 'url'):
                scraped_data = self.scraper.scrape_product_page(product.url)
                if scraped_data:
                    # Update product with scraped data
                    product.description = scraped_data.get('description', product.description)
                    product.image_url = scraped_data.get('images', [None])[0]
                    product.attributes.update(scraped_data.get('attributes', {}))
                    
            enriched_products.append(product)
            
        return enriched_products
    
    def create_vector_store(
        self,
        products: List[Product]
    ) -> FAISS:
        """Create vector store for semantic search"""
        # Create documents from products
        documents = []
        
        for product in products:
            # Create searchable content
            content = f"""
            Product: {product.name}
            Category: {product.category}
            Description: {product.description}
            Price: ${product.price}
            Brand: {product.brand}
            """
            
            metadata = {
                "product_id": product.id,
                "category": product.category,
                "price": product.price,
                "language": product.language
            }
            
            doc = Document(page_content=content, metadata=metadata)
            documents.append(doc)
            
        # Create embeddings and vector store
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
        )
        
        vector_store = FAISS.from_documents(documents, embeddings)
        
        return vector_store

## 9. API Interface

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional, List, Dict, Any
import uvicorn

# API Models
class ChatRequest(BaseModel):
    user_id: str
    message: str
    session_id: Optional[str] = None
    
class ChatResponse(BaseModel):
    response: str
    recommendations: List[Dict[str, Any]]
    session_id: str
    
class RecommendationRequest(BaseModel):
    user_id: Optional[str] = None
    product_id: Optional[str] = None
    n_recommendations: int = 5
    
class RecommendationResponse(BaseModel):
    recommendations: List[Dict[str, Any]]

# Initialize components
nlp_processor = MultilingualNLPProcessor()
recommendation_engine = HybridRecommendationEngine(nlp_processor)
product_catalog = {}  # Would be loaded from database

# Initialize agent
agent = ConversationalRecommenderAgent(
    nlp_processor=nlp_processor,
    recommendation_engine=recommendation_engine,
    product_catalog=product_catalog
)

# Create FastAPI app
app = FastAPI(title="Conversational Recommender API")

@app.post("/chat", response_model=ChatResponse)
async def chat_endpoint(request: ChatRequest):
    """Handle chat messages"""
    try:
        # Process message
        result = await agent.process_message(
            message=request.message,
            user_id=request.user_id,
            session_state=None  # Would retrieve from session store
        )
        
        # Format recommendations
        formatted_recs = []
        for rec in result.get("recommendations", []):
            formatted_recs.append({
                "product_id": rec["product"].id,
                "name": rec["product"].name,
                "price": rec["product"].price,
                "score": rec["score"],
                "reason": rec["reason"]
            })
            
        return ChatResponse(
            response=result["message"],
            recommendations=formatted_recs,
            session_id=request.session_id or f"session_{request.user_id}"
        )
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/recommend", response_model=RecommendationResponse)
async def recommend_endpoint(request: RecommendationRequest):
    """Get product recommendations"""
    try:
        recs = recommendation_engine.get_hybrid_recommendations(
            user_id=request.user_id,
            product_id=request.product_id,
            n_recommendations=request.n_recommendations
        )
        
        formatted_recs = []
        for product_id, score in recs:
            if product_id in product_catalog:
                product = product_catalog[product_id]
                formatted_recs.append({
                    "product_id": product.id,
                    "name": product.name,
                    "price": product.price,
                    "category": product.category,
                    "score": score
                })
                
        return RecommendationResponse(recommendations=formatted_recs)
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {"status": "healthy"}
```

## 10. Main Execution and Demo

```python
async def main():
    """Main execution function"""
    
    print("=== Conversational Recommender System Demo ===\n")
    
    # 1. Initialize components
    print("1. Initializing components...")
    nlp_processor = MultilingualNLPProcessor()
    
    # 2. Load and process data
    print("2. Loading product data...")
    pipeline = DataPipeline()
    products, campaigns = pipeline.process_newsletter_data("newsletter_data.json")
    
    # 3. Build recommendation engine
    print("3. Building recommendation engine...")
    recommendation_engine = HybridRecommendationEngine(nlp_processor)
    recommendation_engine.build_product_embeddings(products)
    
    # Create product catalog
    product_catalog = {p.id: p for p in products}
    
    # 4. Initialize conversational agent
    print("4. Initializing conversational agent...")
    agent = ConversationalRecommenderAgent(
        nlp_processor=nlp_processor,
        recommendation_engine=recommendation_engine,
        product_catalog=product_catalog
    )
    
    # 5. Demo conversations
    print("\n5. Demo Conversations:")
    
    # Demo 1: English conversation
    print("\n--- Demo 1: English Conversation ---")
    response = await agent.process_message(
        "Hi, I'm looking for a new smartphone",
        user_id="user_001"
    )
    print(f"User: Hi, I'm looking for a new smartphone")
    print(f"Agent: {response['message']}")
    
    # Demo 2: Chinese conversation
    print("\n--- Demo 2: Chinese Conversation ---")
    response = await agent.process_message(
        "我想买一个新的智能手机",
        user_id="user_002"
    )
    print(f"User: 我想买一个新的智能手机")
    print(f"Agent: {response['message']}")
    
    # 6. Evaluation
    print("\n6. Running evaluation...")
    evaluator = RecommenderSystemEvaluator()
    
    # Generate some test data
    test_interactions = [
        UserInteraction(
            user_id="user_001",
            product_id="prod_001",
            interaction_type="click",
            timestamp=datetime.now(),
            newsletter_id="news_001"
        )
    ]
    
    # Evaluate recommendations
    rec_metrics = evaluator.evaluate_recommendations(
        test_interactions,
        recommendation_engine
    )
    
    # Generate report
    all_metrics = {
        "recommendation_metrics": rec_metrics,
        "conversation_metrics": {
            "avg_quality": 4.2,
            "avg_relevance": 4.5,
            "avg_helpfulness": 4.3
        },
        "language_metrics": {
            "en": {"language_detection_accuracy": 0.95},
            "zh": {"language_detection_accuracy": 0.92}
        }
    }
    
    evaluator.generate_evaluation_report(all_metrics)
    
    print("\n=== Demo Complete ===")

# Run the demo
if __name__ == "__main__":
    import asyncio
    asyncio.run(main())
```

## Architecture Summary

The conversational recommender system consists of:

### 1. **Data Pipeline Components**
- Web scraper for product enrichment
- Newsletter data processor
- Multi-format data ingestion

### 2. **NLP Processing**
- Multilingual support (Chinese, Thai, Vietnamese, English)
- Intent classification using mDeBERTa
- Entity extraction with XLM-RoBERTa
- Semantic embeddings with multilingual MPNet

### 3. **Recommendation Engine**
- Hybrid approach (content-based + collaborative filtering)
- FAISS for efficient similarity search
- SVD for collaborative filtering
- Personalization based on interaction history

### 4. **Conversational Agent**
- LangGraph-based workflow orchestration
- State management for conversation flow
- Memory for context retention
- Multilingual response generation

### 5. **Evaluation Framework**
- Recommendation metrics (Precision@k, Recall@k, NDCG@k)
- Conversation quality assessment using LLM judges
- Multilingual performance evaluation
- Comprehensive reporting with visualizations

### 6. **API Layer**
- FastAPI endpoints for chat and recommendations
- Session management
- Async processing for scalability

The system is designed to handle the specific requirements of Southeast Asian e-commerce, with robust multilingual support, scalable architecture, and comprehensive evaluation metrics.